In [3]:
%%file test.py
# Test.
from constant import *
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
import sys
import time
import cPickle
import argparse
import math

#test_dataset = ['0', '10', '20', '30', '40', '50', '60', '70', '80', '90', '100']
#test_dataset = ['0', '20', '40', '60', '80', '100']
#### Parameters ####
parser = argparse.ArgumentParser()
parser.add_argument('--gpu', default=0, required=False)
parser.add_argument('--model_type_str', required=True)
parser.add_argument('--model_name', required=True)
parser.add_argument('--test_type_str', required=True)
parser.add_argument('--test_name', required=True)

args = parser.parse_args()
gpu = int(args.gpu)
model_type_str = args.model_type_str
model_name = args.model_name
test_type_str = args.test_type_str
test_name = args.test_name
'''

gpu = 0
model_type_str = 'nocrop'
model_name = '0'
test_type_str = 'nocrop'
test_name = '20'
'''
####################

import caffe

caffe.set_device(gpu)
caffe.set_mode_gpu()

print 'Processing: finetune_alexnet_{}_{} on GPU {}, test on {}'.format(model_type_str, model_name, gpu, test_type_str)

test_net = caffe.Net(result_root + 'model/finetune_alexnet_{}_{}/test_{}_{}.prototxt'\
                .format(model_type_str, model_name, test_type_str, test_name),
                result_root + 'model/finetune_alexnet_{}_{}/finetune_alexnet_{}_{}.caffemodel'\
                .format(model_type_str, model_name, model_type_str, model_name),
                caffe.TEST)

image_sum = 0
test_file = open('{}dataset/test_{}_{}.txt'.format(imagenet_root, test_type_str, test_name), 'r')
image_sum = len(test_file.readlines())
test_file.close()

test_iters = int(math.ceil(image_sum / 256.0)) # batch_size
print test_iters, image_sum

accuracy = 0.0
for i in range(test_iters):
    test_net.forward()
    accuracy += test_net.blobs['accuracy_test'].data
    print '[{}/{}] {}'.format(i, test_iters, accuracy / (i + 1))
accuracy /= test_iters
print 'accuracy: ', accuracy
    
with open(result_root + 'test/accuracy_{}_{}_{}_{}.pickle'.format(model_type_str, model_name, test_type_str, test_name), 'wb') as f:
    cPickle.dump(accuracy, f)

Overwriting test.py


In [2]:
# Generate test_{}_{}.prototxt for testing.
import os
from constant import *

model_type_strs = ['crop_image']
model_names = ['all']
test_type_strs = ['crop', 'crop_img', 'crop_obj', 'crop_image', 'aperture']
test_names = ['0', '10', '20', '30', '40', '50', '60', '70', '80', '90', '100']

for model_type_str in model_type_strs:
    for model_name in model_names:
        f = open(result_root + 'model/finetune_alexnet_{}_{}/train_val.prototxt'.format(model_type_str, model_name))
        lines = f.readlines()
        f.close()
        for test_type_str in test_type_strs:
            for test_name in test_names:
                lines[34] = '    source: "/data2/haow3/data/imagenet/lmdb/test_{}_{}"\n'.format(test_type_str, test_name)
                test_file = open(result_root + 'model/finetune_alexnet_{}_{}/test_{}_{}.prototxt'\
                                 .format(model_type_str, model_name, test_type_str, test_name), 'w')
                test_file.writelines(lines)
                test_file.close()